# Material Designer

This notebook demonstrates the process of sending user-defined parameter values to a parameterized analysis and receive the corresponding simulation output via a Workbench service on a local machine.

In [1]:
import os
import pathlib

In [2]:
from ansys.workbench.core import launch_workbench

Launch the Workbench service on the local machine, using some options.
Define several directories that will be used during the session.
`workdir` is set to the parent directory of the current file.
`assets`, `scripts`, and `wbpz` are subdirectories within the working directory.
The `launch_workbench` function is called to start a Workbench session with specified directory.

In [3]:
workdir = pathlib.Path("__file__").parent

In [4]:
assets = workdir / "assets"

In [5]:
wb = launch_workbench(client_workdir=str(workdir.absolute()), use_insecure_connection=True)

Using insecure connection is not recommended. Please see the documentation for your installed product for additional information.


C:\Users\ansys\actions-runner\_work\pyworkbench-examples\pyworkbench-examples\.venv\Lib\site-packages\ansys\tools\common\cyberchannel.py:187: UserWarning: Starting gRPC client without TLS on localhost:53895. This is INSECURE. Consider using a secure connection.
  warn(f"Starting gRPC client without TLS on {target}. This is INSECURE. Consider using a secure connection.")


Upload the project files to the server using the `upload_file_from_example_repo` method.
The file to upload is `MatDesigner.wbpz`.

In [6]:
wb.upload_file_from_example_repo('material-designer-workflow/wbpz/MatDesigner.wbpz')

Uploading MatDesigner.wbpz:   0%|          | 0.00/261k [00:00<?, ?B/s]

Uploading MatDesigner.wbpz: 100%|██████████| 261k/261k [00:00<00:00, 9.53MB/s]

Execute a Workbench script (`project.wbjn`) to define the project and load the geometry using the `run_script_file` method.
The `set_log_file` method is used to direct the logs to `wb_log_file.log`.

In [7]:
log_path = 'wblog.txt'
wb.set_log_file(log_path)
sys_name = wb.run_script_file(str(assets / "project.wbjn"), log_level='info')

Prepare the Workbench command template to make modifications to the material property, in this case the Young's modulus of the material

In [8]:
wbjn_template = """designPoint1 = Parameters.GetDesignPoint(Name="0")
parameter1 = Parameters.GetParameter(Name="P1")
designPoint1.SetParameterExpression(
    Parameter=parameter1,
    Expression="{} [Pa]")
UpdateAllDesignPoints(DesignPoints=[designPoint1])
"""

Update the project with a new value for the Young's modulus

In [9]:
my_command = wbjn_template.format( 1.6e10 )
wb.run_script_string( my_command )

{}

Extract output values. First, we prepare the Workbench script to quiry output parameter values

In [10]:
extract_output = '''import json
p = Parameters.GetParameter(Name="P{}")
my_tag = p.DisplayText
wb_script_result =json.dumps( my_tag + ',' + str(p.Value) )
'''

Get updated output values

In [11]:
outputs = {}
for p in range( 2 , 12 ):
    return_val = wb.run_script_string( extract_output.format( p ) ).split(',')
    name = return_val[0]
    parameter_val = float(return_val[1])
    outputs[ name ] = parameter_val
print( outputs )

{'E1': 3610891.5343014905, 'E2': 3610888.248890364, 'E3': 799999805.9074863, 'G12': 898955.9754686318, 'G23': 157509429.4833215, 'G31': 157509429.4833215, 'nu12': 0.991244397077384, 'nu13': 0.0013540847304926538, 'nu23': 0.0013540834984631697, 'rho': 392.49992225934625}


Finally, call the `exit` method on the Workbench client to gracefully shut down the service.

In [12]:
wb.exit()